In [6]:
#1.get case name
from glob import glob
import os 
TEST_DIR = "/home/cuixing/MDP/Data_Predict/Test_Data"
case_ids = sorted({os.path.basename(f).split('_')[0] for f in glob(os.path.join(TEST_DIR, "*.nii.gz"))})
print(case_ids)

['CF5008', 'CF5009', 'CF5015', 'CF5017', 'CF6008', 'CF6011', 'CF6028', 'CF6031', 'CF7011', 'CM5115', 'CM6116', 'CM7104', 'CM7109', 'CM7130']


In [7]:
#2.get gt file path
SRC_ROOT = "/nfs/turbo/coe-mreedsensitive/Processing/Chest/TotalSegmentator_FineTuning/SK"
DST_ROOT = "/home/cuixing/MDP/Ground_Truth"


In [10]:
#3. whether exclude some parts
#exclude_files = ["costal_cartilages.nii.gz", "sternum.nii.gz"]
exclude_files = []

In [12]:
#4. merge together
import nibabel as nib
import numpy as np
# IF want all label into that same, mark as True
#MERGE_TO_SINGLE_LABEL = False
MERGE_TO_SINGLE_LABEL = True

def merge_one_case(case):
    gt_folder = os.path.join(SRC_ROOT, case, "segmentations")
    save_dir  = os.path.join(DST_ROOT, case)
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, "merged_gt.nii.gz")

    if not os.path.isdir(gt_folder):
        print(f"[WARN] {case}: {gt_folder} not exist, jump")
        return False

    nii_files = sorted(glob(os.path.join(gt_folder, "*.nii.gz")))
    nii_files = [f for f in nii_files if not any(ex in os.path.basename(f) for ex in exclude_files)]
    if not nii_files:
        print(f"[WARN] {case}: segmentations no nii.gz file，jump")
        return False

    first_img = nib.load(nii_files[0])
    shape = first_img.get_fdata().shape
    affine = first_img.affine
    header = first_img.header

    merged_data = np.zeros(shape, dtype=np.int16)

    for fp in nii_files:
        img = nib.load(fp)
        data = img.get_fdata().astype(np.int16)
        if data.shape != shape:
            raise ValueError(f"{case}: voxel not same size：{os.path.basename(fp)} {data.shape} vs {shape}")

        if MERGE_TO_SINGLE_LABEL:
            merged_data[data > 0] = 1
        else:
            merged_data[data > 0] = data[data > 0]

        print(f"[{case}] merged {os.path.basename(fp)}")

    merged_img = nib.Nifti1Image(merged_data, affine=affine, header=header)
    nib.save(merged_img, save_path)
    print(f"[OK] {case} -> {save_path}")
    return True

# run
ok = err = 0
print(f"Found {len(case_ids)} cases from {TEST_DIR}: {case_ids}")
for case in case_ids:
    try:
        ok += 1 if merge_one_case(case) else 0
    except Exception as e:
        err += 1
        print(f"[ERROR] {case}: {e}")

print(f"Done. success={ok}, errors={err}")

Found 14 cases from /home/cuixing/MDP/Data_Predict/Test_Data: ['CF5008', 'CF5009', 'CF5015', 'CF5017', 'CF6008', 'CF6011', 'CF6028', 'CF6031', 'CF7011', 'CM5115', 'CM6116', 'CM7104', 'CM7109', 'CM7130']
[CF5008] merged costal_cartilages.nii.gz
[CF5008] merged rib_left_1.nii.gz
[CF5008] merged rib_left_10.nii.gz
[CF5008] merged rib_left_11.nii.gz
[CF5008] merged rib_left_12.nii.gz
[CF5008] merged rib_left_2.nii.gz
[CF5008] merged rib_left_3.nii.gz
[CF5008] merged rib_left_4.nii.gz
[CF5008] merged rib_left_5.nii.gz
[CF5008] merged rib_left_6.nii.gz
[CF5008] merged rib_left_7.nii.gz
[CF5008] merged rib_left_8.nii.gz
[CF5008] merged rib_left_9.nii.gz
[CF5008] merged rib_right_1.nii.gz
[CF5008] merged rib_right_10.nii.gz
[CF5008] merged rib_right_11.nii.gz
[CF5008] merged rib_right_12.nii.gz
[CF5008] merged rib_right_2.nii.gz
[CF5008] merged rib_right_3.nii.gz
[CF5008] merged rib_right_4.nii.gz
[CF5008] merged rib_right_5.nii.gz
[CF5008] merged rib_right_6.nii.gz
[CF5008] merged rib_right_7

[CF7011] merged rib_left_3.nii.gz
[CF7011] merged rib_left_4.nii.gz
[CF7011] merged rib_left_5.nii.gz
[CF7011] merged rib_left_6.nii.gz
[CF7011] merged rib_left_7.nii.gz
[CF7011] merged rib_left_8.nii.gz
[CF7011] merged rib_left_9.nii.gz
[CF7011] merged rib_right_1.nii.gz
[CF7011] merged rib_right_10.nii.gz
[CF7011] merged rib_right_11.nii.gz
[CF7011] merged rib_right_12.nii.gz
[CF7011] merged rib_right_2.nii.gz
[CF7011] merged rib_right_3.nii.gz
[CF7011] merged rib_right_4.nii.gz
[CF7011] merged rib_right_5.nii.gz
[CF7011] merged rib_right_6.nii.gz
[CF7011] merged rib_right_7.nii.gz
[CF7011] merged rib_right_8.nii.gz
[CF7011] merged rib_right_9.nii.gz
[CF7011] merged sternum.nii.gz
[OK] CF7011 -> /home/cuixing/MDP/Ground_Truth/CF7011/merged_gt.nii.gz
[CM5115] merged costal_cartilages.nii.gz
[CM5115] merged rib_left_1.nii.gz
[CM5115] merged rib_left_10.nii.gz
[CM5115] merged rib_left_11.nii.gz
[CM5115] merged rib_left_12.nii.gz
[CM5115] merged rib_left_2.nii.gz
[CM5115] merged rib_left

In [14]:
#Check

path = "CF6011"
gt_path = os.path.join("/home/cuixing/MDP/Ground_Truth", path, "merged_gt.nii.gz")
#gt_path = os.path.join("/home/cuixing/MDP/Ground_Truth", path, "segmentations/rib_left_1.nii.gz")
print(f"GT directory: {gt_path}")

img = nib.load(gt_path)
Ground_data = img.get_fdata()
Ground_data = Ground_data.astype(np.int16)  

labels = np.unique(Ground_data)
print("Unique labels in the segmentation:", labels)


GT directory: /home/cuixing/MDP/Ground_Truth/CF6011/merged_gt.nii.gz
Unique labels in the segmentation: [0 1]


In [1]:
# import os
# import nibabel as nib
# import numpy as np
# from glob import glob

# path = "CF6027"

# nii_files = sorted(glob(os.path.join(gt_folder, "*.nii.gz")))

# exclude_files = ["costal_cartilages.nii.gz", "sternum.nii.gz"]
# nii_files = [f for f in nii_files if not any(exclude_name in f for exclude_name in exclude_files)]


# first_img = nib.load(nii_files[0])
# shape = first_img.get_fdata().shape
# affine = first_img.affine
# header = first_img.header

# merged_data = np.zeros(shape, dtype=np.int16)

# for nii_file in nii_files:
#     img = nib.load(nii_file)
#     data = img.get_fdata().astype(np.int16)
#     merged_data[data > 0] = data[data > 0]  # 

#     #print(f"Merged {nii_file}")

# merged_img = nib.Nifti1Image(merged_data, affine=affine, header=header)
# nib.save(merged_img, save_path)
# print(f"Saved merged binary ground truth to: {save_path}")


Merged /home/cuixing/MDP/Ground_Truth/CF6027/segmentations/rib_left_1.nii.gz
Merged /home/cuixing/MDP/Ground_Truth/CF6027/segmentations/rib_left_10.nii.gz
Merged /home/cuixing/MDP/Ground_Truth/CF6027/segmentations/rib_left_11.nii.gz
Merged /home/cuixing/MDP/Ground_Truth/CF6027/segmentations/rib_left_12.nii.gz
Merged /home/cuixing/MDP/Ground_Truth/CF6027/segmentations/rib_left_2.nii.gz
Merged /home/cuixing/MDP/Ground_Truth/CF6027/segmentations/rib_left_3.nii.gz
Merged /home/cuixing/MDP/Ground_Truth/CF6027/segmentations/rib_left_4.nii.gz
Merged /home/cuixing/MDP/Ground_Truth/CF6027/segmentations/rib_left_5.nii.gz
Merged /home/cuixing/MDP/Ground_Truth/CF6027/segmentations/rib_left_6.nii.gz
Merged /home/cuixing/MDP/Ground_Truth/CF6027/segmentations/rib_left_7.nii.gz
Merged /home/cuixing/MDP/Ground_Truth/CF6027/segmentations/rib_left_8.nii.gz
Merged /home/cuixing/MDP/Ground_Truth/CF6027/segmentations/rib_left_9.nii.gz
Merged /home/cuixing/MDP/Ground_Truth/CF6027/segmentations/rib_right_1.ni